In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [2]:
right = pd.read_csv('20cm-try1-pilot_test/3-right_leg_tibi_20250429_190737_820.csv', skiprows=10, sep=",")
left = pd.read_csv('20cm-try1-pilot_test/2-left_leg_tibia_20250429_190737_819.csv', skiprows=10, sep=",")
pelvis = pd.read_csv('20cm-try1-pilot_test/1-pelvis(s2)_20250429_190737_811.csv', skiprows=10, sep=",")

In [8]:
right.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PacketCounter   464 non-null    int64  
 1   SampleTimeFine  464 non-null    int64  
 2   dq_W            464 non-null    float64
 3   dq_X            464 non-null    float64
 4   dq_Y            464 non-null    float64
 5   dq_Z            464 non-null    float64
 6   dv[1]           464 non-null    float64
 7   dv[2]           464 non-null    float64
 8   dv[3]           464 non-null    float64
 9   Status          464 non-null    int64  
dtypes: float64(7), int64(3)
memory usage: 36.4 KB


In [6]:
right.head()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
0,1,1384981139,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2,1384997806,1.0,0.000256,-0.000260,-0.000065,0.160661,-0.029274,-0.008165,0
2,3,1385014473,1.0,0.000300,-0.000292,-0.000062,0.160749,-0.029365,-0.008515,0
3,4,1385031140,1.0,0.000291,-0.000310,-0.000065,0.161394,-0.029261,-0.007908,0
4,5,1385047807,1.0,0.000356,-0.000302,-0.000075,0.161656,-0.029710,-0.008048,0


In [12]:
right['dv[3]'].describe()

count    464.000000
mean      -0.023362
std        0.084664
min       -1.030902
25%       -0.023054
50%       -0.018739
75%       -0.003429
max        0.575202
Name: dv[3], dtype: float64

In [4]:
left.describe()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
count,463.000000,4.630000e+02,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000
mean,232.000000,1.388831e+09,0.997733,0.000503,-0.000346,0.000186,0.157740,0.001086,-0.025781,4.423326
std,133.800847,2.230059e+06,0.046472,0.010752,0.009653,0.002349,0.149767,0.078304,0.167975,67.228579
min,1.000000,1.384981e+09,0.000000,-0.089058,-0.059259,-0.014073,-0.585263,-0.791055,-1.958981,0.000000
25%,116.500000,1.386906e+09,0.999994,-0.000044,-0.000607,0.000029,0.160525,-0.010364,-0.025441,0.000000
50%,232.000000,1.388831e+09,1.000000,0.000208,-0.000258,0.000197,0.161815,0.001630,-0.019810,0.000000
75%,347.500000,1.390756e+09,1.000000,0.000526,0.000375,0.000433,0.163123,0.005774,-0.008561,0.000000
max,463.000000,1.392681e+09,1.000000,0.070062,0.065687,0.023860,1.738278,0.597044,1.361667,1024.000000


In [5]:
pelvis.describe()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
count,463.000000,4.630000e+02,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.0
mean,232.000000,1.388831e+09,0.997821,0.000270,0.000066,-0.000082,0.158822,0.000958,0.034893,0.0
std,133.800847,2.230059e+06,0.046473,0.001945,0.005760,0.001211,0.092902,0.019787,0.052879,0.0
min,1.000000,1.384981e+09,0.000000,-0.009235,-0.033616,-0.007377,-0.134059,-0.168638,-0.329105,0.0
25%,116.500000,1.386906e+09,0.999996,-0.000151,-0.000298,-0.000291,0.159753,0.000477,0.016840,0.0
50%,232.000000,1.388831e+09,1.000000,0.000198,-0.000010,-0.000079,0.161562,0.002378,0.027926,0.0
75%,347.500000,1.390756e+09,1.000000,0.000612,0.000547,0.000088,0.162953,0.004089,0.031700,0.0
max,463.000000,1.392681e+09,1.000000,0.014374,0.070714,0.007108,1.190327,0.106289,0.582826,0.0
